In [1]:
import pandas as pd
# import numpy as np
import json

In [2]:
# reference:
# http://www.southsudanmedicaljournal.com/archive/2008-05/how-to-read-a-chest-x-ray-a-step-by-step-approach.html
step_by_step_inspection_checklist = '''Step 1 Airway:
Check trachea position, carina angle (60–100 degrees), and main stem bronchi.
Look for tubes, pacemaker wires, lines, and foreign bodies. Ensure correct positioning of endotracheal tubes.

Step 2 Bones and Soft Tissues:
Assess clavicles, ribs, thoracic spine, and humerus for fractures or lesions.
Examine soft tissues for subcutaneous air, foreign bodies, and surgical clips.
Be cautious of nipple shadows; compare both sides.

Step 3 Cardiac:
Evaluate heart size, borders, and for pneumomediastinum.
Inspect aorta for widening or calcification, heart valves, and major veins.

Step 4 Diaphragm:
Compare right to left hemidiaphragm height. Consider effusion or paralysis if discrepancies are noted.
Check for free air under the diaphragm in upright positions.

Step 5 Effusion:
Look for blunting of costophrenic angles and fissure tracking.
Examine pleura for thickening, loculations, calcifications, and pneumothorax.

Step 6 Lung Fields:
Identify infiltrates, noting their location and pattern (interstitial vs. alveolar).
Look for air bronchograms, tram tracking, nodules, Kerley B lines, and assess apices.

Step 7 Gastric Air Bubble:
Verify position, check for hiatus hernia, free air, and misplaced bowel loops.

Step 8 Hilum:
Evaluate position, size, lymph nodes, calcified nodules, mass lesions, and pulmonary artery size.
'''

In [3]:
old_prompt = 'AUTHOR: Jacobs, Kathleen \nCLINICAL HISTORY: Chest pain\nCONTEXT: Outpatient\nAGE: 69\nSEX: Female\nBased on AUTHOR and CLINICAL HISTORY, suppose you were a radiologist on X-RAY,  could you provide a detailed report from this chest X-ray?'
old_prompt.split(',')

['AUTHOR: Jacobs',
 ' Kathleen \nCLINICAL HISTORY: Chest pain\nCONTEXT: Outpatient\nAGE: 69\nSEX: Female\nBased on AUTHOR and CLINICAL HISTORY',
 ' suppose you were a radiologist on X-RAY',
 '  could you provide a detailed report from this chest X-ray?']

# Change Finetune Prompts

In [4]:
data = None
with open('../raymond/030424patient_finding_impression.json', 'r') as old_json:
    data = json.load(old_json)
new_qa = []
for item in data:
    new_clue = 'following the X-ray reading procedures outlined as follows whenever you are not certain:\n\n' + step_by_step_inspection_checklist
    new_qa_item = item
    old_prompt_segments = item['conversations'][0]['value'].split(', ')
    old_question = old_prompt_segments[-1].strip()[:-1]
    old_question += ', which should closely resemble the style of the given AUTHOR?'
    old_prompt_segments[-1] = new_clue
    new_qa_item['conversations'][0]['value'] = ', '.join(old_prompt_segments) + '\n' + old_question
    new_qa.append(new_qa_item)
with open('../raymond/030824patient_finding_impression.json', 'w') as new_json:
    json.dump(new_qa, new_json, indent=4)

# Change Inference Prompts

In [5]:
with open('../raymond/030824_test_patient_finding_impression.jsonl', 'w') as new_jsonl:
    with open('../raymond/030424_test_patient_finding_impression.jsonl', 'r') as old_jsonl:
        for line in old_jsonl:
            new_clue = 'following the X-ray reading procedures outlined as follows whenever you are not certain:\n\n' + step_by_step_inspection_checklist
            old_object = json.loads(line)
            new_object = old_object
            old_prompt_segments = old_object['text'].split(', ')
            old_question = old_prompt_segments[-1].strip()[:-1]
            old_question += ', which should closely resemble the style of the given AUTHOR?'
            old_prompt_segments[-1] = new_clue
            new_object['text'] = ', '.join(old_prompt_segments) + '\n' + old_question
            modified_line = json.dumps(new_object)
            new_jsonl.write(modified_line + '\n')

In [2]:
df = pd.read_csv('../luke_scratch/03-04-24/data/test_inference.csv')
df.head()

,id,phonetic_id,author,llava_findings,llava_impression,llava_report,radiologist_report,radiologist_findings,radiologist_impression,similarity,impression_similarity,findings_similarity,is_si_l,is_si_r,lf_len,li_len,rf_len,ri_len,llava_rep_summary,radiologist_rep_summary
0,54059849,Abifel,"Jacobs, Kathleen",FINDINGS:\nLungs are well expanded. Bronchial ...,IMPRESSION: No acute findings.,FINDINGS:\nLungs are well expanded. Bronchial ...,FINDINGS:\nLungs are well expanded and clear. ...,FINDINGS:\nLungs are well expanded and clear. ...,IMPRESSION: No acute findings.,0.911282,0.924824,0.906267,False,False,220,30,184,30,FDG PET/CT demonstrates well-expanded bilatera...,FDG PET/CT demonstrates no evidence of FDG avi...
1,53428634,Aburik,"Hahn, Lewis",FINDINGS:\nLines and Tubes: Right upper extrem...,IMPRESSION: Right upper extremity PICC termina...,FINDINGS:\nLines and Tubes: Right upper extrem...,FINDINGS:\nLines and Tubes: Right upper extrem...,FINDINGS:\nLines and Tubes: Right upper extrem...,IMPRESSION: Right upper extremity PICC termina...,0.726511,0.378502,0.856207,False,False,255,171,328,86,[1] Unchanged right upper extremity PICC termi...,[1] Unchanged right upper extremity PICC termi...
2,53439132,Achoroos,"Smitaman, Edward",FINDINGS:\nPlease see Impression.\n\n\n,"IMPRESSION: Clear lungs. No pneumothorax, pleu...",FINDINGS:\nPlease see Impression.\n\n\nIMPRESS...,FINDINGS:\nLines and Tubes: None Mediastinum:...,FINDINGS:\nLines and Tubes: None Mediastinum:...,IMPRESSION: No acute pulmonary findings.,0.579788,0.608229,0.174392,True,False,35,119,325,40,No evidence of sarcoidosis or other acute abno...,[1] No acute pulmonary findings. [2] Improved ...
3,50219723,Ackoofdek,"Wyatt, Alvin",FINDINGS:\nSee impression.\n\n\n,IMPRESSION: No acute cardiopulmonary abnormali...,FINDINGS:\nSee impression.\n\n\nIMPRESSION: No...,FINDINGS:\nSee impression\n\nIMPRESSION: Exam ...,FINDINGS:\nSee impression\n\n,IMPRESSION: Exam is somewhat limited by patien...,0.379651,0.294585,0.872348,True,True,28,83,26,325,No acute abnormality seen.,[1] No evidence of FDG avid malignancy. [2] Se...
4,52165792,Adefoup,"Weihe, Elizabeth",FINDINGS:/: \nThe cardiac silhouette is enla...,NaN,FINDINGS:/: \nThe cardiac silhouette is enla...,FINDINGS:/: \nNo evidence for focal pneumoni...,FINDINGS:/: \nNo evidence for focal pneumoni...,NaN,0.566880,1.000000,0.566880,False,False,284,0,332,0,[1] Enlarged cardiac silhouette. [2] Superimpo...,FDG PET/CT demonstrates no evidence of focal p...


In [6]:
albert = df[df['author'].str.contains('Hsiao')]
albert = albert.sort_values(by='similarity', ascending=False)
albert.head(50)

,id,phonetic_id,author,llava_findings,llava_impression,llava_report,radiologist_report,radiologist_findings,radiologist_impression,similarity,impression_similarity,findings_similarity,is_si_l,is_si_r,lf_len,li_len,rf_len,ri_len,llava_rep_summary,radiologist_rep_summary
716,52456010,Hasetur,"Hsiao, Albert",FINDINGS:\nUnremarkable cardiomediastinal silh...,IMPRESSION: No acute cardiopulmonary findings.,FINDINGS:\nUnremarkable cardiomediastinal silh...,FINDINGS:\nUnremarkable cardiomediastinal silh...,FINDINGS:\nUnremarkable cardiomediastinal silh...,IMPRESSION: No acute cardiopulmonary findings.,0.981425,0.954470,0.983285,False,False,148,46,147,46,No acute cardiopulmonary findings.,No acute cardiopulmonary findings.
1750,52496876,Sapimor,"Hsiao, Albert",FINDINGS:\nUnremarkable cardiomediastinal silh...,IMPRESSION: No acute cardiopulmonary findings.,FINDINGS:\nUnremarkable cardiomediastinal silh...,FINDINGS:\nUnremarkable cardiomediastinal silh...,FINDINGS:\nUnremarkable cardiomediastinal silh...,IMPRESSION: No acute cardiopulmonary findings.,0.981425,0.954471,0.983285,False,False,148,46,147,46,No acute cardiopulmonary findings.,No acute cardiopulmonary findings.
37,52259451,Arugam,"Hsiao, Albert",FINDINGS:\nUnremarkable cardiomediastinal silh...,IMPRESSION: No acute cardiopulmonary findings.,FINDINGS:\nUnremarkable cardiomediastinal silh...,FINDINGS:\nUnremarkable cardiomediastinal silh...,FINDINGS:\nUnremarkable cardiomediastinal silh...,IMPRESSION: No acute cardiopulmonary findings.,0.981425,0.954471,0.983285,False,False,148,46,147,46,No acute cardiopulmonary findings.,No acute cardiopulmonary findings.
1330,53936593,Nocohou,"Hsiao, Albert",FINDINGS:\nUnremarkable cardiomediastinal silh...,IMPRESSION: No acute cardiopulmonary findings.,FINDINGS:\nUnremarkable cardiomediastinal silh...,FINDINGS:\nUnremarkable cardiomediastinal silh...,FINDINGS:\nUnremarkable cardiomediastinal silh...,IMPRESSION: No acute cardiopulmonary findings.,0.981425,0.954471,0.983285,False,False,150,46,149,46,No acute cardiopulmonary findings.,No acute cardiopulmonary findings.
1000,52483773,Kebleque,"Hsiao, Albert",FINDINGS:\nUnremarkable cardiomediastinal silh...,IMPRESSION: No acute cardiopulmonary findings.,FINDINGS:\nUnremarkable cardiomediastinal silh...,FINDINGS:\nUnremarkable cardiomediastinal silh...,FINDINGS:\nUnremarkable cardiomediastinal silh...,IMPRESSION: No acute cardiopulmonary findings.,0.981425,0.954471,0.983285,False,False,148,46,147,46,No acute cardiopulmonary findings.,No acute cardiopulmonary findings.
2050,52552958,Ustpetom,"Hsiao, Albert",FINDINGS:\nUnremarkable cardiomediastinal silh...,IMPRESSION: No acute cardiopulmonary findings.,FINDINGS:\nUnremarkable cardiomediastinal silh...,FINDINGS:\nUnremarkable cardiomediastinal silh...,FINDINGS:\nUnremarkable cardiomediastinal silh...,IMPRESSION: Normal chest x-ray. No acute cardi...,0.958684,0.775397,0.983285,False,False,148,46,147,66,No acute cardiopulmonary findings.,Unremarkable chest x-ray. No acute abnormaliti...
1504,53372955,Pokayun,"Hsiao, Albert",FINDINGS:\nSee impression.\n\n\n,IMPRESSION: Stable appearance of the chest and...,FINDINGS:\nSee impression.\n\n\nIMPRESSION: St...,FINDINGS:\nSee impression.\n\nIMPRESSION: Stab...,FINDINGS:\nSee impression.\n\n,IMPRESSION: Stable appearance of the chest and...,0.919717,0.902226,0.843004,True,True,28,210,27,225,[1] Stable appearance of the chest and support...,[1] Stable appearance of the chest and support...
518,53371682,Fisoho,"Hsiao, Albert",FINDINGS:\nSee impression.\n\n\n,IMPRESSION: Stable appearance of the chest and...,FINDINGS:\nSee impression.\n\n\nIMPRESSION: St...,FINDINGS:\nSee impression.\n\nIMPRESSION: Stab...,FINDINGS:\nSee impression.\n\n,IMPRESSION: Stable appearance of the chest and...,0.911301,0.891187,0.843004,True,True,28,185,27,215,[1] Stable appearance of the chest and support...,[1] Stable appearance of the chest and support...
1320,54136080,Niheeloch,"Hsiao, Albert",FINDINGS:\nSee impression.\n\n\n,IMPRESS

In [11]:
print(albert.loc[329, 'llava_report'])

FINDINGS:
Unremarkable cardiomediastinal silhouette.  No consolidation, pleural effusion or pneumothorax.  No acute osseous abnormality identified.


IMPRESSION: No acute cardiopulmonary findings.


In [12]:
print(albert.loc[329, 'radiologist_report'])

FINDINGS:
Unremarkable cardiomediastinal silhouette.  No consolidation, pleural effusion or pneumothorax.  Basilar reticular fibrotic changes are noted, unchanged.  No acute osseous abnormality identified.

IMPRESSION: Stable chest x-ray. No acute cardiopulmonary findings.


In [4]:
df1 = pd.read_csv('../luke_scratch/03-08-24/data/test_inference.csv')
df2 = pd.read_csv('../luke_scratch/03-03-24/data/test_inference.csv')
df1.head()

,id,phonetic_id,author,llava_findings,llava_impression,llava_report,radiologist_report,radiologist_findings,radiologist_impression,similarity,impression_similarity,findings_similarity,is_si_l,is_si_r,lf_len,li_len,rf_len,ri_len,llava_rep_summary,radiologist_rep_summary
0,54059849,Abifel,"Jacobs, Kathleen",FINDINGS:\nLines and Tubes: Endotracheal tube ...,IMPRESSION: No evidence of acute traumatic inj...,FINDINGS:\nLines and Tubes: Endotracheal tube ...,FINDINGS:\nLungs are well expanded and clear. ...,FINDINGS:\nLungs are well expanded and clear. ...,IMPRESSION: No acute findings.,0.349926,0.393295,0.303769,False,False,351,63,184,30,[1] Tubes are in place for the patient's trach...,FDG PET/CT demonstrates no evidence of FDG avi...
1,53428634,Aburik,"Hahn, Lewis",FINDINGS:\nLines and Tubes: None Mediastinum:...,IMPRESSION: Mild pulmonary edema.,FINDINGS:\nLines and Tubes: None Mediastinum:...,FINDINGS:\nLines and Tubes: Right upper extrem...,FINDINGS:\nLines and Tubes: Right upper extrem...,IMPRESSION: Right upper extremity PICC termina...,0.482706,-0.066707,0.592814,False,False,199,33,328,86,[1] Unchanged mild pulmonary edema. [2] No new...,[1] Unchanged right upper extremity PICC termi...
2,53439132,Achoroos,"Smitaman, Edward",FINDINGS:\nPlease see Impression.\n\n\n,IMPRESSION: No acute pulmonary abnormality.,FINDINGS:\nPlease see Impression.\n\n\nIMPRESS...,FINDINGS:\nLines and Tubes: None Mediastinum:...,FINDINGS:\nLines and Tubes: None Mediastinum:...,IMPRESSION: No acute pulmonary findings.,0.549578,0.900780,0.174392,True,False,35,43,325,40,No acute pulmonary abnormality.,[1] No acute pulmonary findings. [2] Improved ...
3,50219723,Ackoofdek,"Wyatt, Alvin",FINDINGS:\nSee impression.\n\n\n,IMPRESSION: 1.\nNo acute cardiopulmonary disea...,FINDINGS:\nSee impression.\n\n\nIMPRESSION: 1....,FINDINGS:\nSee impression\n\nIMPRESSION: Exam ...,FINDINGS:\nSee impression\n\n,IMPRESSION: Exam is somewhat limited by patien...,0.565210,0.513437,0.872348,True,True,28,161,26,325,No evidence of acute cardiopulmonary disease.,[1] No evidence of FDG avid malignancy. [2] Se...
4,52165792,Adefoup,"Weihe, Elizabeth",FINDINGS:\nPlease see impression.\n,IMPRESSION: No acute cardiopulmonary abnormali...,FINDINGS:\nPlease see impression.\nIMPRESSION:...,FINDINGS:/: \nNo evidence for focal pneumoni...,FINDINGS:/: \nNo evidence for focal pneumoni...,NaN,0.372395,-0.081017,0.182622,True,False,33,150,332,0,No acute abnormality. Endotracheal tube termin...,FDG PET/CT demonstrates no evidence of focal p...


In [23]:
print(df.loc[2, 'llava_report'])
print('---')
print(df1[df1['lf_len'] <= 50].loc[2, 'llava_report'])

FINDINGS:
Please see Impression.


IMPRESSION: Clear lungs. No pneumothorax, pleural effusion, or pulmonary edema. Cardiomediastinal silhouette is normal.
---
FINDINGS:
Please see Impression.


IMPRESSION: No acute pulmonary abnormality.


In [24]:
print(df.loc[2, 'radiologist_report'])
print('---')
print(df1[df1['lf_len'] <= 50].loc[2, 'radiologist_report'])

FINDINGS:
Lines and Tubes: None  Mediastinum: The cardiomediastinal silhouette is unchanged.  Lungs: Improved lung volumes compared to prior. Bibasal linear opacities likely representing atelectasis.  Pleura: No pneumothorax or effusion.  Bones and soft tissues: Unchanged             Preliminary created by: Thumar, Vishal

IMPRESSION: No acute pulmonary findings.
---
FINDINGS:
Lines and Tubes: None  Mediastinum: The cardiomediastinal silhouette is unchanged.  Lungs: Improved lung volumes compared to prior. Bibasal linear opacities likely representing atelectasis.  Pleura: No pneumothorax or effusion.  Bones and soft tissues: Unchanged             Preliminary created by: Thumar, Vishal

IMPRESSION: No acute pulmonary findings.


In [31]:
same_ex = df1[(df1['id'] == 52456010) & (df1['phonetic_id'] == 'Hasetur')]
same_ex

,id,phonetic_id,author,llava_findings,llava_impression,llava_report,radiologist_report,radiologist_findings,radiologist_impression,similarity,impression_similarity,findings_similarity,is_si_l,is_si_r,lf_len,li_len,rf_len,ri_len,llava_rep_summary,radiologist_rep_summary
716,52456010,Hasetur,"Hsiao, Albert",FINDINGS:\nSee impression.\n\n\n,IMPRESSION: Stable appearance of the chest and...,FINDINGS:\nSee impression.\n\n\nIMPRESSION: St...,FINDINGS:\nUnremarkable cardiomediastinal silh...,FINDINGS:\nUnremarkable cardiomediastinal silh...,IMPRESSION: No acute cardiopulmonary findings.,0.402623,0.143559,0.31754,True,False,28,147,147,46,[1] Stable appearance of the chest and support...,No acute cardiopulmonary findings.


In [38]:
print(df.loc[716, 'llava_report'])
print('---')
print(df.loc[716, 'radiologist_report'])

FINDINGS:
Unremarkable cardiomediastinal silhouette.
No consolidation, pleural effusion or pneumothorax.
No acute osseous abnormality identified.


IMPRESSION: No acute cardiopulmonary findings.
---
FINDINGS:
Unremarkable cardiomediastinal silhouette.
No consolidation, pleural effusion or pneumothorax.
No acute osseous abnormality identified.

IMPRESSION: No acute cardiopulmonary findings.


In [33]:
print(same_ex.loc[716, 'llava_report'])
print('---')
print(same_ex.loc[716, 'radiologist_report'])

FINDINGS:
See impression.


IMPRESSION: Stable appearance of the chest and supportive devices.
Endotracheal tube terminates 4.5 cm above the carina.
Stable interstitial edema.
---
FINDINGS:
Unremarkable cardiomediastinal silhouette.
No consolidation, pleural effusion or pneumothorax.
No acute osseous abnormality identified.

IMPRESSION: No acute cardiopulmonary findings.


In [37]:
print(df2[(df2['id'] == 52456010) & (df2['phonetic_id'] == 'Hasetur')].loc[716, 'llava_report'])
print('---')
print(df2[(df2['id'] == 52456010) & (df2['phonetic_id'] == 'Hasetur')].loc[716, 'radiologist_report'])

FATIGUE: No acute cardiopulmonary abnormality.

IMPRESSION: No acute cardiopulmonary abnormality.
---
FINDINGS:
Unremarkable cardiomediastinal silhouette.
No consolidation, pleural effusion or pneumothorax.
No acute osseous abnormality identified.

IMPRESSION: No acute cardiopulmonary findings.
